In [1]:
import iisignature
import pandas as pd
import numpy as np

from datetime import datetime

from Backtesting.Create_dataframes import CreateDataframe

from Brokers.Alpaca.Alpaca_keyes import API_KEY, SECRET_KEY
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, LSTM

2025-01-31 16:32:15.812632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Normalize data and instead of predictiong five numbers, just predict one over the 5 days

In [2]:
bank_account = 10000
commission_fee = 0.01
slippage_cost = 0.1

create_df = CreateDataframe()

start_date_train = datetime(2023, 1, 1) 
end_date_train = datetime(2025, 1, 1)

data = create_df.Alpaca(
    api_key=API_KEY,
    secret_key=SECRET_KEY,
    start_date=start_date_train,
    end_date=end_date_train,
    interval=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),
    asset_list="TSLA"
)

In [3]:
data['date'] = data['timestamp'].dt.date  # Extract the date part
unique_days = {day: i for i, day in enumerate(sorted(data['date'].unique()))}
data['day'] = data['date'].map(unique_days)

In [4]:
output_data = data.groupby("day").agg(
    close=("close", "last")
)

output_data["return"] = output_data["close"].pct_change(periods=5) #periods=n
output_data["state"] = np.where(output_data["return"] >= 0, 1, -1)

In [5]:
def leadlag(X, use_time = False):
   if (not use_time):
       lead = []
       lag = []
       for val_lag, val_lead in zip(X[:-1], X[1:]):
           lead.append(val_lag)
           lag.append(val_lag)
           lead.append(val_lead)
           lag.append(val_lag)
       lead.append(X[-1])
       lag.append(X[-1])
       return np.c_[lead, lag]
   else:
       lead = []
       lag = []
       t = []
       time = 0
       t.append(time)
       lag.append(X[0])
       lead.append(X[0])
       for val_lag, val_lead in zip(X[:-1], X[1:]):
           time += 1
           t.append(time)
           lead.append(val_lag)
           lag.append(val_lag)
           t.append(time)
           lead.append(val_lead)
           lag.append(val_lag)
           t.append(time)
           lead.append(val_lead)
           lag.append(val_lead)
       return np.c_[lead, lag, t]

In [6]:
# Prepare input
X_list = []

n_days = len(output_data)
for i in range(n_days - 30 - 5):
    window = data[(data["day"] >= i)&(data["day"] < i + 30)]
    path = window["close"].to_numpy()
    ll_path = leadlag(path)
    x = iisignature.sig(ll_path, 10)
    X_list.append(x.reshape(1, -1))

X = np.vstack(X_list)

In [7]:
y_list = []

for i in range(31, n_days-4):
    window = output_data[(output_data.index >= i)&(output_data.index < i + 5)]
    res = window["state"].to_numpy()[-1] # .reshape(1, -1)
    y_list.append(res)

y = np.vstack(y_list)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
mu = np.mean(X_train)
sigma2 = np.std(X_train)

X_train = (X_train - mu) / sigma2
X_test = (X_test - mu) / sigma2

In [35]:
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="tanh"))

In [36]:
print(X_train.shape[1])

2046


In [37]:
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=60, batch_size=32)

Epoch 1/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0191 - loss: 1.0254      
Epoch 2/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0610 - loss: 1.0111 
Epoch 3/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0620 - loss: 1.0009 
Epoch 4/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0606 - loss: 0.9982 
Epoch 5/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0512 - loss: 1.0376 
Epoch 6/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0562 - loss: 0.9727 
Epoch 7/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0485 - loss: 0.9689 
Epoch 8/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0360 - loss: 0.9807 
Epoch 9/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0532 - loss: 0.9496 
Epoch 10/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0711 - loss: 0.9356 
Epoch 11/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0714 - loss: 0.9320 
Epoch 12/60
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accur

In [38]:
pred = model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step 


In [39]:
print(pred)

[[-0.97970814]
 [ 0.06515748]
 [ 0.06757198]
 [ 0.04278886]
 [ 0.07348257]
 [ 0.06611477]
 [ 0.02587664]
 [ 0.06781615]
 [ 0.05404393]
 [-0.39549646]
 [-0.5797886 ]
 [ 0.06121916]
 [ 0.05579532]
 [ 0.05374987]
 [-0.68652815]
 [ 0.06683671]
 [ 0.18311624]
 [ 0.06565211]
 [ 0.0205707 ]
 [ 1.        ]
 [ 0.05769522]
 [ 0.06781457]
 [ 1.        ]
 [ 0.06376   ]
 [ 0.06753631]
 [ 0.81764317]
 [ 0.9970899 ]
 [ 1.        ]
 [ 0.06027663]
 [ 0.06725264]
 [-0.21700233]
 [ 0.06558643]
 [ 0.05762975]
 [ 0.00900199]
 [ 0.07078669]
 [-0.30146825]
 [ 0.05943384]
 [ 0.07812159]
 [ 0.9999909 ]
 [ 0.06749102]
 [-0.00586793]
 [ 0.02054969]
 [ 0.06817452]
 [ 0.06716318]
 [-0.61797655]
 [-0.6990355 ]
 [ 1.        ]
 [ 0.04161856]
 [-0.52321804]
 [ 0.067549  ]
 [ 0.05897683]
 [ 0.07649443]
 [-0.92270684]
 [ 0.06061395]
 [ 0.02280493]
 [-0.73565775]
 [ 0.05741199]
 [ 1.        ]
 [ 0.06637125]
 [ 0.06743866]
 [-0.8492505 ]
 [ 0.06450608]
 [ 0.02816685]
 [ 0.06488132]
 [ 1.        ]
 [ 0.03905158]
 [ 0.06650

In [40]:
print(np.abs(pred - y_test) < 0.6)

[[ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 